In [1]:
#ALL IMPORTS
import numpy as np
import pandas as pd

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
# READING DATA
season17_18 = pd.read_csv("17_18.csv")
season18_19 = pd.read_csv("18_19.csv")
season19_20 = pd.read_csv("19_20.csv")

In [3]:
class Football:
    
    def __init__(self, data, name):
        self.data=data
        self.cleanData()
        self.name=name
        self.teams = set(self.data.HomeTeam) | set(self.data.AwayTeam)
#         self.teams = {i for i in self.data.HomeTeam}
#         self.teams.update({i for i in self.data.AwayTeam})
    
    def getTeamWins(self):
        winTeams = dict()
        for i in self.teams:
            winTeams[i]=0
        
        for i in range(len(self.data)):
            if self.data.FTR[i] == 'A': #Away team won
                winTeams[self.data.AwayTeam[i]]+=1
            elif self.data.FTR[i] == 'H': #Home team won
                winTeams[self.data.HomeTeam[i]]+=1
        
        return winTeams
    
    def getTeamLoses(self):
        loseTeams = dict()
        for i in self.teams:
            loseTeams[i]=0
        
        for i in range(len(self.data)):
            if self.data.FTR[i] == 'A': #Away team won
                loseTeams[self.data.HomeTeam[i]]+=1
            elif self.data.FTR[i] == 'H': #Home team won
                loseTeams[self.data.AwayTeam[i]]+=1
        
        return loseTeams
        
        return None
    
    def getTeamDraws(self):
        drawTeams = dict()
        for i in self.teams:
            drawTeams[i] = 0
            
        for i in range(len(self.data)):
            if self.data.FTR[i] == 'D':
                drawTeams[self.data.HomeTeam[i]] += 1
                drawTeams[self.data.AwayTeam[i]] += 1
        
        return drawTeams
    
    def getTeamGames(self): #How many games were played by each team
        gamesCount = dict()
        for i in self.teams:
            gamesCount[i] = 0
        for i in range(len(self.data)):
            gamesCount[self.data.HomeTeam[i]]+=1
            gamesCount[self.data.AwayTeam[i]]+=1
        return gamesCount
    
    def getTeamPoints(self):
        points = dict()
        #WIN 3 POINTS
        #DRAW 1 POINT
        for i in self.teams:
            points[i] = 0
        for i in self.getTeamWins():
            points[i]=self.getTeamWins()[i]*3
        for i in self.getTeamDraws():
            points[i]+=self.getTeamDraws()[i]*1
        return points
        
    
    def getTeamsTotalGoals(self):
        teamTotalGoals = dict()
        for i in self.teams:
            teamTotalGoals[i] = 0
            
        for i in range(len(self.data)):
            teamTotalGoals[self.data.HomeTeam[i]] += self.data.FTHG[i]
            teamTotalGoals[self.data.AwayTeam[i]] += self.data.FTAG[i]
            
        return teamTotalGoals
    
    def cleanData(self):
        colums = ["HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR", "HTHG", "HTAG", "HTR", "Referee", "HS", "AS", "HST", "AST",
                 "HF", "AF", "HC", "AC", "HY", "AY", "HR", "AR", "B365H", "B365D", "B365A"]
        for col in self.data.columns:
            contains = False
            for j in colums:
                if col == j:
                    contains = True
            if not contains:
                self.data = self.data.drop(col, axis=1)
    
    def getWrongPrediction(self):
        #TODO find matches were odds and match results were not equal
        luckyLoser = dict()
        referees = dict()
        goalsDifference = []
        homeOrAway={"Home":0,"Away":0}
        for i in self.teams:
            luckyLoser[i] = 0
        for i in self.data.Referee:
            referees[i]=0
        
        kokku=0
        for i in range(len(self.data)):
            homeOdd = float(self.data.B365H[i])
            drawOdd = float(self.data.B365D[i])
            awayOdd = float(self.data.B365A[i])
            
            homeGoals = int(self.data.FTHG[i])
            awayGoals = int(self.data.FTAG[i])
            
            if homeOdd == awayOdd: #If odds are same, then there is no right prediction
                continue
            if homeOdd < awayOdd and self.data.FTR[i]!='H' and self.data.FTR[i]!='D':
                referees[self.data.Referee[i]]+=1
                #print("Home:",homeOdd,"Draw:",drawOdd,"Away:",awayOdd,self.data.HomeTeam[i],"VS",self.data.AwayTeam[i],"WON SECOND TEAM")
                luckyLoser[self.data.AwayTeam[i]]+=1
                homeOrAway['Away']+=1
                goalsDifference.append(abs(homeGoals-awayGoals))
                kokku+=1
            elif homeOdd > awayOdd and self.data.FTR[i]!='A' and self.data.FTR[i]!='D':
                referees[self.data.Referee[i]]+=1
                #print("Home:",homeOdd,"Draw:",drawOdd,"Away:",awayOdd,self.data.HomeTeam[i],"VS",self.data.AwayTeam[i],"WON FIRST TEAM")
                luckyLoser[self.data.HomeTeam[i]]+=1
                homeOrAway['Home']+=1
                goalsDifference.append(abs(homeGoals-awayGoals))
                kokku+=1
        print(self.name)
        #print(kokku/len(self.data)*100,"%")
        #print(sorted(luckyLoser.items(), key=lambda x: x[1], reverse=True))
        print("REFEREES")
        print(sorted(referees.items(), key=lambda x: x[1], reverse=True))
        print("Where was wrong prediction",homeOrAway)
        print("Goals difference,",goalsDifference)
        print()
                
    
    def showTabel(self):
        wins=[i for i in self.getTeamWins().values()]
        loses=[i for i in self.getTeamLoses().values()]
        draws=[i for i in self.getTeamDraws().values()]
        points=[i for i in self.getTeamPoints().values()]
        games=[i for i in self.getTeamGames().values()]
        tabel = {'Teams': list(self.teams),
                 'Wins': wins,
                 'Loses': loses,
                 'Draws': draws,
                 'Points': points,
                 'Games': games}
        df = pd.DataFrame(tabel)
        df=df.sort_values(by=['Points'],ascending=False)
        df=df.to_string(index=False)
        print(df)

In [4]:
season1718 = Football(season17_18,"Season 17_18")
season1819 = Football(season18_19,"Season 18_19")
season1920 = Football(season19_20,"Season 19_20")
#season1718.showTabel()
#season1718.getWrongPrediction()
#season1819.getWrongPrediction()
#season1920.getWrongPrediction()

In [5]:
#CREATING PREDICTION MODEL

predictingData = season17_18[["HomeTeam","AwayTeam","B365H","B365D","B365A"]]
predictingData=predictingData.append(season18_19[["HomeTeam","AwayTeam","B365H","B365D","B365A"]],ignore_index = True)
predictingData=predictingData.append(season19_20[["HomeTeam","AwayTeam","B365H","B365D","B365A"]],ignore_index = True)

for i in range(len(predictingData)):
    home = float(predictingData.B365H[i])
    draw = float(predictingData.B365D[i])
    away = float(predictingData.B365A[i])
    
    beforeSort = [home,draw,away]
    afterSort = sorted(beforeSort)
    
    idx1=afterSort.index(beforeSort[0])
    idx2=afterSort.index(beforeSort[1])
    idx3=afterSort.index(beforeSort[2])
    
    predictingData.iat[i,2]=idx1
    predictingData.iat[i,3]=idx2
    predictingData.iat[i,4]=idx3
    
df = pd.DataFrame(columns=['accuracy', 'n_estimators', 'max_depth'])    
    
for i in range(5,220):
    for j in range(1,int(i*3/5)):
        X_train = pd.get_dummies(predictingData, columns=["HomeTeam","AwayTeam"])
        y_train = season17_18['FTR']
        y_train = y_train.append(season18_19['FTR'])
        y_train = y_train.append(season19_20['FTR'])
        rf = RandomForestClassifier(n_estimators=i, max_depth=j, random_state=0).fit(X_train, y_train)
        x_train, x_test, y_train, y_test = train_test_split(X_train,y_train, train_size=0.75)
        clrTree = rf.fit(x_train, y_train)
        outTree = clrTree.predict(x_test)
        a=pd.DataFrame([[accuracy_score(y_test, outTree)*100,i,j]],columns=['accuracy', 'n_estimators', 'max_depth'])
        df=df.append(a,ignore_index=True)
        #print("Accuracy for Decision Tree Classifier: " + str(accuracy_score(y_test, outTree)*100)+"%"," ",i)


In [6]:
print(df.sort_values(by=['accuracy'], ascending=False))

        accuracy n_estimators max_depth
13634  64.573991          216         4
1237   64.125561           67         4
3448   64.125561          110         5
4628   63.677130          127         5
1658   63.677130           77        10
231    63.677130           30        12
2604   63.677130           96         2
179    63.677130           27         6
1568   63.228700           75         8
1018   63.228700           61         5
1051   63.228700           62         3
39     63.228700           14         4
2900   63.228700          101        11
1020   62.780269           61         7
11782  62.780269          201         3
585    62.780269           47         2
820    62.780269           55         6
887    62.331839           57         9
3010   62.331839          103         2
73     62.331839           18         6
8391   62.331839          170        10
3981   62.331839          118         4
641    62.331839           49         4
2127   62.331839           87         4
